In [27]:
%pip -q install google-genai

In [26]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [28]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
# Link Google Calendar
from urllib.parse import quote_plus
import datetime

def gerar_link_google_calendar(titulo, horario_inicio_str, horario_fim_str):
    """
    Gera um link para adicionar um evento ao Google Calendar.

    Args:
        titulo (str): O título do evento.
        horario_inicio_str (str): O horário de início do evento no formato HH:MM.
        horario_fim_str (str): O horário de fim do evento no formato HH:MM.

    Returns:
        str: O link para o Google Calendar ou None em caso de erro na formatação do horário.
    """
    try:
        data_hoje = datetime.date.today()
        data_hoje_str = data_hoje.strftime("%Y%m%d")
        inicio_fmt = data_hoje_str + "T" + horario_inicio_str.replace(":", "") + "00"
        fim_fmt = data_hoje_str + "T" + horario_fim_str.replace(":", "") + "00"
    except ValueError:
        return None

    base = "https://www.google.com/calendar/render?action=TEMPLATE"
    text = f"&text={quote_plus(titulo)}"
    dates = f"&dates={quote_plus(inicio_fmt)}/{quote_plus(fim_fmt)}"
    details = "&details=Agendamento%20de%20Exame" # Personalizei o detalhe
    timezone = "&ctz=America/Sao_Paulo"
    return f"{base}{text}{dates}{details}{timezone}"

if __name__ == '__main__':
    titulo_exame = "Consulta de Retorno - Cardiologia"
    horario_inicio = "10:30"
    horario_fim = "11:00"

    link_calendario = gerar_link_google_calendar(titulo_exame, horario_inicio, horario_fim)

    if link_calendario:
        print(f"Link para adicionar ao Google Calendar: {link_calendario}")
    else:
        print("Erro ao gerar o link do Google Calendar.")

Link para adicionar ao Google Calendar: https://www.google.com/calendar/render?action=TEMPLATE&text=Consulta+de+Retorno+-+Cardiologia&dates=20250516T103000/20250516T110000&details=Agendamento%20de%20Exame&ctz=America/Sao_Paulo


In [ ]:
lista_exames = []

def coletar_dados_exame():
    """Coleta os dados do exame inseridos pelo usuário."""
    dados = {}
    dados['tipo_exame'] = input("Digite o tipo do exame: ")
    dados['data_exame'] = input("Digite a data do exame (AAAA-MM-DD): ")
    dados['nome_medico'] = input("Digite o nome do médico (opcional, deixe em branco se não houver): ")
    dados['nome_laboratorio'] = input("Digite o nome do laboratório (opcional, deixe em branco se não houver): ")
    print("Pressione Enter para arquivar o exame.")
    input() # Simula o botão de envio.
    return dados

def arquivar_exame(dados_exame):
    """
    Recebe um dicionário com os dados do exame e o adiciona à lista de exames.
    Simula uma mensagem de confirmação.
    """
    lista_exames.append(dados_exame)
    print(f"Exame de {dados_exame.get('tipo_exame', 'desconhecido')} arquivado com sucesso.")

def buscar_exames(criterios_busca):
    """
    Recebe um dicionário com critérios de busca e retorna uma lista de exames correspondentes.
    A busca é realizada com base nos critérios fornecidos.
    """
    resultados = []
    if criterios_busca:
        for exame in lista_exames:
            correspondencia = True
            for chave, valor_busca in criterios_busca.items():
                if valor_busca:  # Verifica se o valor de busca não está vazio
                    if chave == 'periodo':
                        if isinstance(valor_busca, str) and valor_busca.isdigit() and len(valor_busca) == 4:
                            ano_exame = exame.get('data_exame', '').split('-')[0]
                            if ano_exame != valor_busca:
                                correspondencia = False
                                break
                        # Adicione aqui lógica para outros formatos de período, se necessário
                    elif exame.get(chave) != valor_busca:
                        correspondencia = False
                        break
            if correspondencia:
                resultados.append(exame)
        if not resultados:
            print("\nNenhum exame encontrado com os critérios fornecidos.")
    else:
        print("Nenhum critério de busca fornecido.")
    return resultados

if __name__ == '__main__':
    while True:
        print("\nOpções:")
        print("1 - Adicionar novo exame")
        print("2 - Buscar exames")
        print("3 - Listar todos os exames")
        print("4 - Sair")

        opcao = input("Escolha uma opção: ")

        if opcao == '1':
            dados_novo_exame = coletar_dados_exame()
            arquivar_exame(dados_novo_exame)
        elif opcao == '2':
            criterios = {}
            print("\nSelecione o critério de busca (ou pressione Enter para buscar todos):")
            print("1 - Tipo do exame")
            print("2 - Data do exame")
            print("3 - Nome do médico")
            print("4 - Nome do laboratório")
            print("5 - Período (ano)")
            escolha_busca = input("Digite o número da sua escolha: ")

            if escolha_busca == '1':
                criterios['tipo_exame'] = input("Digite o tipo do exame: ")
            elif escolha_busca == '2':
                criterios['data_exame'] = input("Digite a data do exame (AAAA-MM-DD): ")
            elif escolha_busca == '3':
                criterios['nome_medico'] = input("Digite o nome do médico: ")
            elif escolha_busca == '4':
                criterios['nome_laboratorio'] = input("Digite o nome do laboratório: ")
            elif escolha_busca == '5':
                criterios['periodo'] = input("Digite o período (ano AAAA): ")
            elif not escolha_busca:
                pass # Buscar todos os exames se Enter for pressionado
            else:
                print("Opção de busca inválida.")

            resultados_busca = buscar_exames(criterios)
            if resultados_busca:
                print("\nResultados da busca:")
                for exame in resultados_busca:
                    print(exame)
            elif criterios: # Se o usuário escolheu um critério mas não houve resultados
                pass # A mensagem já é impressa dentro da função buscar_exames
        elif opcao == '3':
            if lista_exames:
                print("\nLista de todos os exames:")
                for exame in lista_exames:
                    print(exame)
            else:
                print("\nAinda não há exames arquivados.")
        elif opcao == '4':
            print("Saindo do programa.")
            break
        else:
            print("Opção inválida. Tente novamente.")


Opções:
1 - Adicionar novo exame
2 - Buscar exames
3 - Listar todos os exames
4 - Sair
Escolha uma opção: 4
Saindo do programa.


In [2]:
!pip install Flask
!pip install flask_ngrok

In [9]:
# Este é um exemplo básico para demonstrar como você poderia interagir com elementos HTML
# usando Python no Colab. Note que Python rodando no Colab não interage diretamente
# com a renderização do HTML no navegador da mesma forma que JavaScript faria.
# O foco aqui é em como você poderia processar dados ou simular interações.

# Para realmente manipular o DOM (Document Object Model) de uma página web renderizada,
# você precisaria usar JavaScript no lado do cliente (dentro do navegador).

# No entanto, podemos usar Python para tarefas como:
# 1. Simular o recebimento de dados de um formulário HTML.
# 2. Processar esses dados.
# 3. Gerar HTML dinamicamente (se você estivesse construindo a página com Python).

# Exemplo 1: Simulação de envio de mensagem
def enviar_mensagem(mensagem):
  """Simula o envio de uma mensagem."""
  print(f"Mensagem a ser enviada: {mensagem}")
  # Aqui você poderia adicionar lógica para armazenar a mensagem,
  # enviar para um servidor, etc.

# Simulação de interação com o input de texto
mensagem_digitada = "Olá do Python!"
enviar_mensagem(mensagem_digitada)

# Exemplo 2: Simulação de clique em um botão
def acao_arquivar():
  """Simula a ação de arquivar."""
  print("Ação: Arquivar")
  # Adicione aqui a lógica para arquivar algo.

def acao_agendar():
  """Simula a ação de agendar."""
  print("Ação: Agendar")
  # Adicione aqui a lógica para agendar algo.

def acao_buscar():
  """Simula a ação de buscar."""
  print("Ação: Buscar")
  # Adicione aqui a lógica para realizar uma busca.

# Simulação de cliques nos botões
acao_arquivar()
acao_agendar()
acao_buscar()

# Em um cenário real onde você tem um backend Python (usando frameworks como Flask ou Django),
# você usaria Python para:
# - Receber dados de formulários HTML.
# - Consultar bancos de dados.
# - Realizar lógica de negócios.
# - Renderizar páginas HTML dinamicamente.

# Para interatividade direta no navegador com o HTML que você definiu, JavaScript seria necessário.
# Python no Colab é útil para processamento de dados, análise e tarefas de backend.

Mensagem a ser enviada: Olá do Python!
Ação: Arquivar
Ação: Agendar
Ação: Buscar


In [12]:
import json

def salvar_exames(lista_exames, nome_arquivo="exames.json"):
    with open(nome_arquivo, 'w') as f:
        json.dump(lista_exames, f)

def carregar_exames(nome_arquivo="exames.json"):
    try:
        with open(nome_arquivo, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return []

# ... (seu código de coleta e arquivamento) ...

if __name__ == '__main__':
    lista_exames = carregar_exames() # Carrega os dados no início

    # ... (sua lógica de adicionar e buscar exames) ...

    salvar_exames(lista_exames) # Salva os dados antes de sair ou em momentos específicos

In [14]:
import csv

def salvar_exames_csv(lista_exames, nome_arquivo="exames.csv"):
    with open(nome_arquivo, 'w', newline='') as csvfile:
        fieldnames = lista_exames[0].keys() if lista_exames else []
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(lista_exames)

def carregar_exames_csv(nome_arquivo="exames.csv"):
    try:
        with open(nome_arquivo, 'r', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            return list(reader)
    except FileNotFoundError:
        return []

# ... (seu código de coleta e arquivamento) ...

if __name__ == '__main__':
    lista_exames = carregar_exames_csv() # Carrega os dados no início

    # ... (sua lógica de adicionar e buscar exames) ...

    salvar_exames_csv(lista_exames) # Salva os dados antes de sair ou em momentos específicos

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [17]:
import google.generativeai as genai
from IPython.display import Markdown, display

# Sua chave de API do Gemini
GOOGLE_API_KEY = "SUA_CHAVE_DE_API"
genai.configure(api_key=GOOGLE_API_KEY)

# Define o modelo a ser usado
MODEL_ID = "gemini-pro" # Ou outro modelo de sua preferência

# Configuração da ferramenta de busca do Google
tools = [
    {
        "name": "google_search",
        "tool_function": {
            "name": "google_search",
            "description": "Use a busca do Google para obter informações.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "A consulta de pesquisa.",
                    }
                },
                "required": ["query"],
            },
        },
    }
]

# Inicializa o modelo com as ferramentas
model = genai.GenerativeModel(MODEL_ID, tools=tools)

# Sua pergunta para o Gemini
contents = "Implementação de um chatbot onde o usuario poderá solicitar ao chat para: arquivar os exames médicos em um tabela no google sheets; agendar no google calendario os proximos compromissos de saúde, buscar por exame arquivado, buscar o compromisso agendado. Como posso começar a implementar essas funcionalidades usando a API do Google?"

# Gera a resposta
response = model.generate_content(contents)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

ValueError: Unknown field for FunctionDeclaration: tool_function